# CHSH Game

The **CHSH Game** quantum kata is a series of exercises designed
to get you familiar with the CHSH game.

In it, two players (Alice and Bob) try to win the following game:

Each of them is given a bit (Alice gets X and Bob gets Y), and
they have to return new bits (Alice returns A and Bob returns B)
so that X ∧ Y = A ⊕ B. The trick is, they can not communicate during the game.

* You can read more about CHSH game in the [lecture notes](https://cs.uwaterloo.ca/~watrous/CPSC519/LectureNotes/20.pdf) by
  John Watrous. 
* Q# Samples repository has [an implementation of the CHSH
  game](https://github.com/microsoft/Quantum/tree/master/samples/algorithms/chsh-game)
  that includes an explanation of the history and theory behind the game.

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl/⌘+Enter.


To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [ ]:
%package Microsoft.Quantum.Katas::0.10.1911.1607

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


## Part I. Classical CHSH


### Task 1.1. Win Condition
**Input:** 

  1. Alice and Bob's starting bits (X and Y),

  2. Alice and Bob's output bits (A and B).

**Output:** 
True if Alice and Bob won the CHSH game, that is, if X ∧ Y = A ⊕ B, and false otherwise.

In [ ]:
%kata T11_WinCondition_Test 

function WinCondition (x : Bool, y : Bool, a : Bool, b : Bool) : Bool {
    // ...
    fail "Task 1.1 not implemented yet";
}

### Task 1.2. Alice and Bob's classical strategy

In this task you have to implement two functions, one for Alice's classical strategy and one for Bob's.
Note that they are covered by one test, so you have to implement both to pass the test. Once you implement one of the strategies, execute its cell - it will fail with the error message indicating that the other strategy is not implemented yet. Once you implement the second strategy, execute its cell to get the test result.

**Input:** Alice's starting bit (X).

**Output:** The bit that Alice should output (A) to maximize their chance of winning.

In [ ]:
%kata T12_ClassicalStrategy_Test 

operation AliceClassical (x : Bool) : Bool {
    // ...
    fail "Alice's strategy in task 1.2 not implemented yet";
}

**Input:** Bob's starting bit (Y).

**Output:** The bit that Bob should output (B) to maximize their chance of winning.

In [ ]:
%kata T12_ClassicalStrategy_Test 

operation BobClassical (y : Bool) : Bool {
    // ...
    fail "Bob's strategy in task 1.2 not implemented yet";
}

## Part II. Quantum CHSH

In the quantum version of the game, the players still can not
communicate during the game, but they are allowed to share 
qubits from a Bell pair before the start of the game.


### Task 2.1. Entangled pair

**Input:** An array of two qubits in the $|00\rangle$ state.

**Goal:** Create a Bell state $|\Phi^{+}\rangle = \frac{1}{\sqrt{2}} \big( |00\rangle + |11\rangle \big)$ on these qubits.

In [ ]:
%kata T21_CreateEntangledPair_Test 

operation CreateEntangledPair (qs : Qubit[]) : Unit {
    // ...
    fail "Task 2.1 not implemented yet";
}

### Task 2.2. Alice's quantum strategy

**Inputs:**

  1. Alice's starting bit (X),

  2. Alice's half of Bell pair she shares with Bob.

**Goal:** Measure Alice's qubit in the Z basis if her bit is 0 (false), or the X basis if her bit is 1 (true), and return the result.
The state of the qubit after the operation does not matter.

In [ ]:
%kata T22_AliceQuantum_Test 

operation AliceQuantum (bit : Bool, qubit : Qubit) : Bool {
    // ...
    fail "Task 2.2 not implemented yet";
}

### Task 2.3. Rotate Bob's qubit

**Inputs:**

  1. The direction to rotate: true for clockwise, false for counterclockwise,

  2. Bob's qubit.

**Goal:** Rotate the qubit $\frac{\pi}{8}$ radians around the Y axis in the given direction.

<br/>
<details>
  <summary>Need a hint? Click here</summary>
  Ry operation applies a rotation by a given angle in counterclockwise direction.
</details>

In [ ]:
%kata T23_RotateBobQubit_Test 

open Microsoft.Quantum.Math;

operation RotateBobQubit (clockwise : Bool, qubit : Qubit) : Unit {
    // ...
    fail "Task 2.3 not implemented yet";
}

### Task 2.4. Bob's quantum strategy

**Inputs:**

  1. Bob's starting bit (Y),

  2. Bob's half of Bell pair he shares with Alice.

**Goal:** Measure Bob's qubit in the $\frac{\pi}{8}$ basis if his bit is 0 (false), or the $-\frac{\pi}{8}$ basis if his bit is 1 (true), and return the result.
The state of the qubit after the operation does not matter.

In [ ]:
%kata T24_BobQuantum_Test 

operation BobQuantum (bit : Bool, qubit : Qubit) : Bool {
    // ...
    fail "Task 2.4 not implemented yet";
}

### Task 2.5. Play the CHSH game using the quantum strategy

**Input:**
Operations that return Alice and Bob's output bits (A and B) based on their quantum
strategies and given their respective qubits from the Bell pair.
Alice and Bob have already been told what their starting bits X and Y are.

**Goal:** Return Alice and Bob's output bits (A, B).
Note that this task uses strategies `AliceQuantum` and `BobQuantum`, which you've implemented in tasks 2.2 and 2.4, respectively.

In [ ]:
%kata T25_PlayQuantumCHSH_Test 

operation PlayQuantumCHSH (askAlice : (Qubit => Bool), askBob : (Qubit => Bool)): (Bool, Bool) {
    // ...
    fail "Task 2.5 not implemented yet";
}